In [ ]:
!pip install pandas
import pandas as pd
import numpy as np
import re
from typing import Dict
from pathlib import Path
import io
import warnings

Wczytanie danych

In [ ]:
# --- KONFIGURACJA ---
data_dir = Path('./data')
csv_paths = sorted(data_dir.glob('*.csv'))

# Regex do filtrowania (bez zmian)
mies_regex = re.compile(r'MIES', re.IGNORECASE)
column_filter = lambda col: not mies_regex.search(col)

# --- POPRAWIONA FUNKCJA WCZYTUJĄCA ---
def try_read_csv(path, col_filter=None):
    # ELA zazwyczaj używa utf-8 lub cp1250, a separatorem jest średnik.
    # Zmieniam kolejność: najpierw średnik, potem reszta.
    encodings = ['utf-8', 'utf-8-sig', 'cp1250', 'latin-1']
    seps = [';', '\t', ',', '|']
    
    read_kwargs = {'usecols': col_filter} if col_filter else {}
    last_exc = None

    for enc in encodings:
        for sep in seps:
            try:
                # 1. Próba silnikiem C (szybki)
                df = pd.read_csv(
                    path, 
                    encoding=enc, 
                    sep=sep, 
                    engine='c', 
                    low_memory=False, 
                    on_bad_lines='skip',
                    **read_kwargs
                )
                
                # --- SANITY CHECK (NOWOŚĆ) ---
                # Jeśli df jest pusty LUB ma mniej niż 2 kolumny -> to prawdopodobnie zły separator
                if df.empty or df.shape[1] < 2:
                    raise ValueError(f"Suspicious shape {df.shape} with sep='{sep}'")

                return df, f"{enc}, sep='{sep}', engine=c"

            except Exception as e_c:
                # To nie jest właściwy separator/kodowanie, idziemy dalej
                pass

        # 2. Fallback: Silnik Python z autodetekcją (wolniejszy, inteligentniejszy)
        try:
            df = pd.read_csv(
                path, 
                encoding=enc, 
                sep=None,  # Autodetekcja
                engine='python', 
                on_bad_lines='skip',
                **read_kwargs
            )
            
            if df.empty or df.shape[1] < 2:
                 raise ValueError(f"Suspicious shape {df.shape} with auto-sep")
                 
            return df, f"{enc}, sep=auto, engine=python"
        except Exception:
            pass

    # 3. Ostatnia deska ratunku: wczytanie jako tekst i wymuszenie
    try:
        with open(path, 'rb') as f:
            raw = f.read()
        
        text = raw.decode('utf-8', errors='replace')
        # Próba wymuszenia średnika na "brudnym" tekście
        df = pd.read_csv(
            io.StringIO(text), 
            sep=';', 
            engine='python', 
            on_bad_lines='skip',
            **read_kwargs
        )
        return df, "decoded bytes, forced sep=';'"
    except Exception as e:
        raise ValueError(f"CRITICAL: Could not read file. Last error: {e}")

# --- PĘTLA ŁADUJĄCA ---
loaded = []
print("Rozpoczynam wczytywanie plików z walidacją kształtu danych...")

for p in csv_paths:
    try:
        df, meta = try_read_csv(p, col_filter=column_filter)
        print(f"✅ Read {p.name:<40} | {meta:<30} | shape: {df.shape}")
        loaded.append((p.stem, df))
    except Exception as e:
        print(f"❌ Failed {p.name}: {e}")

dataframes = [df for _, df in loaded]
dataframes_dict = dict(loaded) # Zmieniam nazwę na czystszą

Czyszczenie danych

In [ ]:
# Wykrywanie kolumn datowych
def _looks_like_date_series(s: pd.Series, sample_size: int = 200, threshold: float = 0.5) -> bool:
    """
    Sprawdza czy seria (kolumna) wygląda jak data.
    Szuka wzorców takich jak: 2020-01-15, 01/12/2020, oraz nazw miesięcy
    """
    # Konwertuje do tekstu i usuwa puste wartości
    vals = s.dropna().astype(str)
    if vals.empty:
        return False
    
    # Bierze losową próbkę (max 200 wartości) do analizy
    vals = vals.sample(min(len(vals), sample_size), random_state=0)
    
    # Regex dla dat: szuka liczb rozdzielonych myślnikami/kreskami/spacjami
    # np. 2020-01-15 lub 15/12/2020
    date_regex = re.compile(r'^\s*\d{1,4}(?:[\-\/\.\s]\d{1,2}){1,2}\s*$')
    
    # Regex dla nazw miesięcy (polska + angielska)
    month_names = re.compile(
        r'\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|'
        r'styc|lut|mar|kwi|maj|cze|lip|sie|wrz|paź|lis|gru)\b', 
        re.I
    )
    
    # Sprawdza czy wartości pasują do wzoru daty LUB zawierają nazwę miesiąca
    matches = vals.str.match(date_regex) | vals.str.contains(month_names)
    
    # Jeśli co najmniej 50% wartości to daty, zwróć True
    return matches.sum() >= max(1, int(threshold * len(vals)))

In [ ]:
# Czyszczenie dataframe'u
def clean_df(df: pd.DataFrame, name: str = "", remove_outliers: bool = True) -> tuple:
    """
    Czyści ramkę danych i (opcjonalnie) usuwa outliery.
    Zwraca: (df_po_czyszczeniu, df_przed_usunieciem_outlierow)
    (Blok usuwania outlierów został zakomentowany.)
    """
    # Robimy kopię, żeby nie modyfikować oryginalnego obiektu przekazanego do funkcji.
    df = df.copy()

    # Lista kolumn, które chcemy POMINAĆ przy wykrywaniu outlierów (np. ID, nazwy)
    skip_columns_for_outliers = {
        'p_rok_od', 'p_kierunek_id', 'p_poziom', 'p_forma', 'p_uczelnia_id',
        'p_nazwa_uczelni', 'p_jedn', 'p_nazwa_jedn', 'p_woj', 'p_profil',
        'p_dziedzina_new', 'p_uczelnia_skrot', 'p_poziom_tekst_pl',
        'p_nazwa_kierunku_pelna', 'p_kierunek_nazwa', 'p_spec_nazwa', 'u_uczelnia_id', "u_n", "u_n_wzus","u_n_pozazus", "u_proc_wzus","u_proc_pozazus","u_n_dosw_rekr", "p_n",
        "u_n_dosw_studia", "p_n_wzus",
    }

    # KROK 1: Normalizuje nazwy kolumn
    # Zamienia na małe litery, usuwa spacje, zastępuje je podkreśleniami
    df.columns = [str(c).strip().lower().replace(" ", "_") for c in df.columns]
    # Usuwamy ewentualne duplikaty nazw kolumn (może się zdarzyć przy złym imporcie).
    df = df.loc[:, ~df.columns.duplicated()]

    # KROK 2: Usuwa całkowicie puste kolumny
    # axis=1 oznacza kolumny, how="all" oznacza całkowicie puste
    df.dropna(axis=1, how="all", inplace=True)

    # KROK 3: Naprawia tekstowe kolumny
    # Bierze tylko kolumny typu tekstowego (object)
    obj_cols = df.select_dtypes(include="object").columns.tolist()
    for c in obj_cols:
        # Konwertuj do string type, usuń spacje na początku/końcu, usuń znaki BOM
        df[c] = df[c].astype("string").str.strip().str.replace("\ufeff", "", regex=False)

         # Zamienia typowe znaczniki braku danych na pd.NA (None)
    df.replace({"": pd.NA, "NA": pd.NA, "N/A": pd.NA, "na": pd.NA, "-": pd.NA, "—": pd.NA, "None": pd.NA}, inplace=True)

    # KROK 4: Konwertuje kolumny liczbowe (tekst → liczby)
    for c in df.columns:
        # Jeśli kolumna jest tekstem, spróbuj skonwertować na liczby
        if df[c].dtype == "object" or pd.api.types.is_string_dtype(df[c]):
            # Usuń spacje wewnątrz liczb
            s = df[c].astype("string").str.replace(r"\s+", "", regex=True)
            # Zamień przecinki na kropki (polski format → międzynarodowy)
            s = s.str.replace(",", ".", regex=False)
            # Usuń wszystkie znaki oprócz cyfr, kropki i minusa
            s_clean = s.str.replace(r"[^0-9\.\-]", "", regex=True)
            # Konwertuj na liczby (błędy zamień na NaN)
            coerced = pd.to_numeric(s_clean, errors="coerce")
            non_null_count = coerced.notna().sum()
            # Jeśli co najmniej 30% wartości to liczby (nie puste), zaakceptuj konwersję
            if non_null_count > 0 and non_null_count >= max(1, int(0.3 * len(coerced))):
                df[c] = coerced

    # KROK 5: Konwertuje kolumny datowe (tekst → daty)
    for c in df.columns:
        # Jeśli kolumna jest tekstem
        if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_string_dtype(df[c]):
            # Sprawdź czy wygląda jak data
            if _looks_like_date_series(df[c]):
                # Wyłącz warningi podczas konwersji
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", UserWarning)
                    # Konwertuj na datetime, przyjmując format DD/MM/YYYY
                    parsed = pd.to_datetime(df[c], errors="coerce", dayfirst=True)

                    # Jeśli co najmniej 30% wartości zostało skonwertowane, zaakceptuj
                if parsed.notna().sum() >= max(1, int(0.3 * len(parsed))):
                    df[c] = parsed

    # KROK 6: Usuwa puste wiersze i duplikaty
    # Usuwa wiersze które są całkowicie puste
    df.dropna(axis=0, how="all", inplace=True)
    # Usuwa wiersze które są całkowicie identyczne
    df.drop_duplicates(inplace=True)

    # Zapisujemy snapshot ramki po standardowym czyszczeniu, przed ewentualnym usuwaniem outlierów.
    df_before_outlier_removal = df.copy()

    # KROK 7: Usuwa wartości skrajne (outliers) — metoda IQR
    # POMIJAMY WYBRANE KOLUMNY (takie jak ID, kody, poziomy, kierunki itd.)
    """
    if remove_outliers:
        OUTLIER_QUANTILE = 0.99
        numeric_cols = [c for c in df.select_dtypes(include=[np.number]).columns.tolist()]

        mask_total = pd.Series(False, index=df.index)
        removed_entries = []

        for c in numeric_cols:
            if c in skip_columns_for_outliers:
                continue
            upper = df[c].quantile(OUTLIER_QUANTILE)
            if pd.isna(upper):
                continue
            mask = df[c].notna() & (df[c] > upper)
            if mask.any():
                for idx in df.index[mask]:
                    removed_entries.append((idx, c, df.at[idx, c]))
                mask_total |= mask

        if mask_total.any():
            total_to_remove = int(mask_total.sum())
            print(f"[outliers - percentyl {OUTLIER_QUANTILE}] {name} :: usuwam {total_to_remove} wierszy zawierających TOP {int((1-OUTLIER_QUANTILE)*100)}% wartości (limit 100 wypisanych):")
            shown = 0
            for idx, col, val in removed_entries:
                print(f"  index={idx}  column='{col}'  value={val}")
                shown += 1
                if shown >= 100:
                    remaining = len(removed_entries) - shown
                    if remaining > 0:
                        print(f"  ... oraz {remaining} pozostałych pozycji")
                    break
            df = df.loc[~mask_total].copy()
            print(f"Usunięto {total_to_remove} wierszy.")
    """

    # 8. Reset indeksu — porządkujemy indeksy po ewentualnych usunięciach.
    df.reset_index(drop=True, inplace=True)

    # Zwracamy: (wersja po czyszczeniu, snapshot przed outlierami)
    return df, df_before_outlier_removal

In [ ]:
# Tworzy folder /output w którym będą zapisywane pliki CSV

output_dir = Path("./output")
output_dir.mkdir(parents=True, exist_ok=True)

# KOD DO CZYSZCZENIA DANYCH 
# Stosuje czyszczenie do wszystkich załadowanych df'ów
cleaned_dataframes_dict: Dict[str, pd.DataFrame] = {}

for name, df in dataframes_dict_dirty.items():
    # 1) Wykonaj czyszczenie BEZ usuwania outlierów -> otrzymujemy (cleaned_no_outliers, snapshot_przed_outlierami)
    cleaned_no_outliers, df_before_outliers = clean_df(df, name, remove_outliers=False)
    
    # 2) Wykonaj usunięcie outlierów na już wstępnie oczyszczonym df -> otrzymujemy (cleaned_with_outliers_removed, snapshot_przed_outlierami)
    #    (druga wartość nie jest potrzebna tutaj, bo df_before_outliers już mamy)
    # cleaned_with_outliers_removed, _ = clean_df(cleaned_no_outliers, name, remove_outliers=True)
    
    # Zapisz obie wersje do CSV (bez indeksu, UTF-8)
    safe_name = name.replace(" ", "_")
    before_path = output_dir / f"{safe_name}_before_outliers.csv"
    # after_path = output_dir / f"{safe_name}_after_outliers.csv"
    
    df_before_outliers.to_csv(before_path, index=False, encoding="utf-8")
    # cleaned_with_outliers_removed.to_csv(after_path, index=False, encoding="utf-8")
    
    print(f"Exported: {before_path.name} ({df_before_outliers.shape})")
    
    # Zachowaj końcowy (po outlierach) w słowniku do dalszej analizy
    cleaned_dataframes_dict[name] = df_before_outliers


In [ ]:
# Upewniamy się, że korzystamy ze świeżo wczytanych danych
# (lista 'loaded' pochodzi z poprawionego skryptu wczytującego)
dataframes_dict = dict(loaded)

print(f"Liczba poprawnych ramek danych do przetworzenia: {len(dataframes_dict)}")

# Tworzymy folder output
output_dir = Path("./output")
output_dir.mkdir(parents=True, exist_ok=True)

# Słownik na oczyszczone dane
cleaned_dataframes_dict = {}

print("Rozpoczynam ponowny eksport na poprawnych danych...")

for name, df in dataframes_dict.items():
    # Zabezpieczenie: jeśli mimo wszystko df jest pusty, pomiń go i zgłoś
    if df.empty:
        print(f"⚠️ OSTRZEŻENIE: Ramka '{name}' jest pusta w pamięci! Pomijam.")
        continue

    try:
        # 1. Wykonaj czyszczenie (korzystamy z funkcji clean_df zdefiniowanej wcześniej)
        # Zwraca: (wersja_czysta, wersja_przed_outlierami)
        cleaned_final, df_snapshot = clean_df(df, name, remove_outliers=False)
        
        # 2. Generowanie nazwy pliku
        safe_name = name.replace(" ", "_")
        before_path = output_dir / f"{safe_name}_processed.csv"
        
        # 3. ZAPIS (Format polski: średnik i przecinek)
        df_snapshot.to_csv(
            before_path, 
            index=False, 
            sep=';', 
            decimal=',', 
            encoding='utf-8-sig'
        )
        
        print(f"✅ Exported: {before_path.name:<45} | Shape: {df_snapshot.shape}")
        
        # Zachowaj w pamięci wersję z kropkami (do obliczeń w Pythonie)
        cleaned_dataframes_dict[name] = df_snapshot
        
    except Exception as e:
        print(f"❌ Error processing {name}: {e}")

print(f"\nZakończono. Pliki w: {output_dir.absolute()}")